In [ ]:
import spacy
import re
import random
from itertools import islice
import pandas as pd
import nltk
nltk.download('propbank')

In [2]:
from nltk.corpus import propbank

In [3]:
nlp = spacy.load('en') # initiate spacy parser for English
datapath = 'q_temps.tsv' 
df = pd.read_csv(datapath, sep = '\t') # load question templates from tsv
df.columns = ['Q', 'POS', 'TAG']
aux_list = ['aux', 'auxpass', 'ROOT', 'advcl', 'ccomp', 'conj']
verb_list = ['VBG', 'VBN', 'VBD', 'VBP', 'VBZ', 'VB']
subj_list = [ 'nsubj', 'nsubjpass', 'expl','csubj','csubjpass']
obj_list = ['obj', 'dobj', 'pobj']
conj_list = ['CC', 'IN']
indefinites = ['a', 'an']
time_list = ['DATE', 'TIME','WRB']
aux_dict = {'VBD': ['did', 'do', 'VBD', 'O', '', 'aux'],'VBP': ['do', 'do', 'VB', 'O', '', 'aux'], 'VBZ': ['does', 'do', 'VBZ', 'O', '', 'aux']}
pro_dict = {'mine': 'yours', 'yours': 'mine', 'ours': 'yours', 'we': 'you', 'We': 'you', 'my': 'your', 'our': 'your', 'your': 'my', 'myself': 'yourself', 'ourself': 'yourself', 'yourself': 'myself', 'me': 'you', 'mine': 'yours', 'yours': 'mine', 'ours': 'yours', 'i': 'you'}
pro_dict2 = {'she': 'her', 'he': 'him', 'they': 'them', 'her': 'her', 'him': 'him', 'them': 'them'}
personal_nouns = ['PERSON', 'ORG']
prep_list = ['at','for','to','on','by','with','in','from','into','about','behind','against','of','as']

In [4]:
pb_inst = propbank.instances()
verb_dict = {pb_inst[i].roleset[:-3]:i for i,item in enumerate(pb_inst)} # get verbs and their position in propbank list

In [5]:
def you_and_me(s): 
    sentlist = s.split()
    for i, word in enumerate(sentlist):
        if word.lower() == 'i' and sentlist[i+1].lower() == 'am':
            sentlist[i] = 'you'
            sentlist[i+1] = 'are'
        elif word.lower() == 'you' and sentlist[i+1].lower() == 'are':
            sentlist[i] = 'I'
            sentlist[i+1] = 'am'
        elif word.lower() == 'i' and sentlist[i+1].lower() == 'was':
            sentlist[i] = 'you'
            sentlist[i+1] = 'were'
    return ' '.join(sentlist)

In [6]:
def return_tags(s):
    # tag words in sentence using spacy
    parsed_s = nlp(s)
    return [[str(X).lower(), X.lemma_, X.tag_, X.ent_iob_, X.ent_type_, X.dep_] if X.ent_iob_ == 'O' else [str(X), X.lemma_, X.tag_, X.ent_iob_, X.ent_type_, X.dep_] for X in parsed_s]

In [7]:
def specify_obj(s):
    qs = []
    for i, tup in enumerate(s):
        if tup[5] in obj_list and not tup[2] == 'PRP':
            tags = [item[5] for item in s[:i+1]]
            pattern = list(zip(range(len(tags)),tags))
            no_pattern = ([tup[1]+str(tup[0]) for tup in pattern])
            pattern_dict = dict(zip(no_pattern,range(len(tags))))
            object_pattern = re.search(r'(?:nummod([0-9]+) )?(?:amod([0-9]+),?(?: cc([0-9]+) conj([0-9]+))* )*(?:compound([0-9]+) )?(?:(?:obj|pobj|dobj)([0-9]+))',' '.join(no_pattern))
            if object_pattern not in [[], None]:
                obj_tags = object_pattern.group().split()
                obj_start = pattern_dict[obj_tags[0]]
                obj_end = pattern_dict[obj_tags[-1]]
                if s[obj_start-1][2] not in ['DT','PRP$']:
                    qs.append('Which ' + ' '.join([word[0] for word in s[obj_start:i+1]]) + ' ' + ' '.join([word[0] for word in s[:obj_start]]) + '?')
                elif s[obj_start-1][0].lower() in indefinites:
                    qs.append('Which ' + ' '.join([word[0] for word in s[obj_start:i+1]]) + ' ' + ' '.join([word[0] for word in s[:obj_start-1]]) + '?')
                    qs.append('What is the ' + ' '.join(word[0] for word in s[obj_start:i+1]) + ' about?')
    return qs

In [8]:
def specify_subj(s):
    qs = []
    for i, tup in enumerate(s):
        if tup[5] in subj_list:
            tags = [item[2] for item in s[:i+1]]
            pattern = list(zip(range(len(tags)),tags))
            no_pattern = ([tup[1]+str(tup[0]) for tup in pattern])
            pattern_dict = dict(zip(no_pattern,range(len(tags))))
            subject_pattern = re.search(r'(?:JJ[R|S]?[0-9]+ )*(?:(?:NNS*|EX)[0-9]+) ?(?:IN[0-9]+ )?(?:NNS*[0-9] ?)*',' '.join(no_pattern))
            if subject_pattern not in [[], None]:
                subj_tags = subject_pattern.group().split()
                subj_start = pattern_dict[subj_tags[0]]
                subj_end = pattern_dict[subj_tags[-1]]
                if s[subj_start-1][2] not in ['DT','PRP$']:
                    qs.append('Which ' + ' '.join([word[0] for word in s[subj_end:]]) + '?')
                elif s[subj_start-1][0].lower() in indefinites:
                    qs.append('Which ' + ' '.join([word[0] for word in s[subj_end:]]) + '?')
    return qs

In [9]:
def set_verb(s):
    # add auxiliary for subject-aux reversal
    for i,tup in enumerate(islice(s, len(s))):
        if tup[5] == 'aux':
            return s
        if tup[2] in aux_dict and tup[1] != 'be':
            tup[0] = tup[1]
            ss = s[:i] + [[x for x in aux_dict[tup[2]]]] + [[y for y in tup]] + s[i+1:]
            return ss
    return s

In [10]:
def reverse_subj_aux(t):
    # find subject and switch subj and aux
    for i, tup in enumerate(islice(t, len(t)-1)):
        if tup[5] in subj_list and t[i+1][5] in aux_list:
            tags = [item[2] for item in t[:i+1]]
            subject_pattern = re.findall(r'(?:DT|PRP\$)? ?(?:JJ[R|S]? )*(?:NN[PS]*|PRP|EX) ?(?:IN )?(?:NN[PS]* ?)*',' '.join(tags))
            if subject_pattern not in [[], None]:
                subj_tags = [tag for tag in subject_pattern[0].split()]
                subj_start = tags.index(subj_tags[0])
                subj_end = tags.index(subj_tags[-1])+1
                new_order = t[:subj_start] + t[i+1:i+2] + t[subj_start:subj_end] + t[i+2:]
                return new_order
    return t

In [11]:
def change_view(s): 
    # switch personal pronouns etc first-second person 
    for word in s:
        if word[2] in ['PRP','PRP$','NNS','NN']:
            if str(word[0]).lower() in pro_dict:
                word[0] = pro_dict[str(word[0]).lower()]
    return s

In [12]:
def why(s):
    q = []
    if s[0][5] in aux_list:
        q.append('Why ' + ' '.join([word[0] if not word[0] in indefinites else 'the' for word in s]) + '?')
    return q

In [13]:
def where(s):
    q = []
    to_delete = []
    if s[0][5] in aux_list:
        for i, word in enumerate(s):
            if word[0] == 'there' and not word[2] == 'EX':
                to_delete.append(i)
    for index in to_delete:
        del s[i]
        q.append('Where exactly ' + ' '.join([word[0] if not word [0] in indefinites else 'the' for word in s]) + '?')
    return q

In [14]:
def when(s):
    q = 'When ' + ' '.join([word[0] if not word[0] in indefinites else 'the' for word in s]) + '?'
    return q

In [15]:
def animacy(s):
    qlist = []
    for word in s: 
        if word[4] in personal_nouns:
            qlist.append('What else do we know about %s?' % word[0])
        elif word[0].lower() in pro_dict2.keys():
            qlist.append('What else do we know about %s?' % pro_dict2[word[0].lower()])
    return qlist

In [16]:
def subcategorization(s):
    qs = []
    defn = ['the', 'the', 'DT', 'O', '', 'det']
    objects = [word for word in s if word[5] in obj_list]
    for word in s:
        if word[2] in verb_list and not word[1] == 'be':
            if word[1] in verb_dict.keys():
                roles = [tup[1] for tup in pb_inst[verb_dict[word[1]]].arguments]
                if 'ARG0' in roles:
                    roles.remove('ARG0')
                if 'ARGM-MNR' in roles:
                    qs.append('How ' + ' '.join([word[0] for word in s]) + '?')
                if len([role for role in roles if role in ['ARG1', 'ARG2']]) > len(objects):
                    qs.append('What ' + ' '.join([word[0] for word in s]) + '?')
                for prep in prep_list:
                    if prep in re.findall(r'-([a-z]+)', ' '.join(roles)):
                        sent_list = [word[0] if not word[0] in indefinites else 'the' for word in s]
                        if not prep in sent_list:
                            if prep == 'at':
                                qs.append('Where ' + ' '.join(sent_list) + '?')
                            else:
                                qs.append('What ' + ' '.join(sent_list) + ' ' + prep + '?')
                                qs.append('Who ' + ' '.join(sent_list) + ' ' + prep + '?')
    return qs

In [17]:
def ne_questions(s):
    nes = []
    s = s[::-1]    #get Multiword Expressions
    pattern = [word[3] for word in s]
    for j,sym in enumerate(pattern):
        if sym == 'B':
            if s[j][2] == 'DT':
                s[j][2] = s[j+1][2]
        if sym == 'I':
            s[j+1][0] = s[j+1][0] + ' ' + s[j][0]
            s[j][4] = ''
    s = s[::-1] 
    tags = {tag[0]: (tag[4],tag[2]) for tag in s if not tag[4] == ''}
    questions = []
    for name, tup in tags.items():
        df_selection = df[(df.TAG == tup[0]) & (df.POS == tup[1])]
        qlist = df_selection['Q']
        qs = []
        for ql in qlist:
            new = ql.replace(tup[0], name)    #replace tag in template with word 
            qs.append(new)
        nes.append(qs)
    return nes

In [18]:
def simplify_nested(s):
    tags = [word[5] for word in s]
    for i, tag in enumerate(tags):
        if tag in subj_list:
            j = i
            commas = 0
            while j < len(tags):
                if tags[j] == 'punct':
                    commas +=1
                elif tags[j] in aux_list and commas%2 == 0:
                    return s[:i+1] + s[j:]
                j += 1
        return s

In [19]:
def separate_matrix(sent):
    s = []
    def separate(s_):
        tags = [word[5] for word in s_]
        if 'mark' not in tags:
            s.append(s_)
        elif 'mark' in tags:
            separate(s_[:tags.index('mark')])
            separate(s_[tags.index('mark')+1:])
    separate(sent)
    return(s)

In [20]:
def split_at_and(sent):
    s = []
    def split(s_):
        tags = [word[2] for word in s_]
        if 'CC' not in tags:
            s.append(s_)
        else:
            sub1, sub2 = s_[:tags.index('CC')], s_[tags.index('CC')+1:]
            if sub1[0][5] in subj_list and any(set(aux_list) & set([w[5] for w in sub1])) and sub2[0][5] in subj_list and any(set(aux_list) & set([w[5] for w in sub2])):
            #if any(set(subj_list) & set([w[5] for w in sub1])) and any(set(aux_list) & set([w[5] for w in sub1])) and any(set(subj_list) & set([w[5] for w in sub2])) and any(set(aux_list) & set([w[5] for w in sub2])):
                split(sub1)
                split(sub2)
            else: 
                s.append(s_)
    split(sent)
    return s

In [21]:
def removal(sent):
    chopped = []
    def remove_adjuncts(s):
        tags = [word[5] for word in s]
        if 'punct' not in tags:
            chopped.append(s)
        else:
            new1, new2 = s[:tags.index('punct')], s[tags.index('punct')+1:]
            for new in [new1, new2]:
                if any(set(subj_list) & set([t[5] for t in new])) and any(set(aux_list) & set([t[5] for t in new])):
                    remove_adjuncts(new)
                else: pass
    remove_adjuncts(sent)
    return chopped

In [22]:
def remove_conj(s): #this is a miscellaneous method that removes all adverbs except for 'there', as well as leading conjunctions and prepositional phrases 
    to_delete = []
    for i, word in enumerate(s):
        if word[5] == 'advmod' and word[2] == 'RB' and not word[0].lower() == 'there':
            to_delete.append(i)
            if s[i-1][2] == ',' and s[i+1][2] == ',':
                to_delete.append(i-1)
    if s[0][2] == 'CC':
        to_delete.append(0)
    if s[0][2] == 'IN':
        deps = [word[5] for word in s]
        subj = [tag for tag in deps if tag in subj_list]
        index = deps.index(subj[0])
        for i in range(index):
            to_delete.append(i)
    new_s = [word for word in s if not s.index(word) in to_delete]
    return new_s

In [23]:
def split_at_and(sent):
    s = []
    def split(s_):
        tags = [word[2] for word in s_]
        if 'CC' not in tags:
            s.append(s_)
        else:
            sub1, sub2 = s_[:tags.index('CC')], s_[tags.index('CC')+1:]
            try:
                if sub1[0][5] in subj_list and any(set(aux_list) & set([w[5] for w in sub1])) and sub2[0][5] in subj_list and any(set(aux_list) & set([w[5] for w in sub2])):
                    split(sub1)
                    split(sub2)
                else: 
                    s.append(s_)
            except IndexError: 
                s.append(s_)
    split(sent)
    return s

In [24]:
def ask_source(s):
    return 'How do you know that %s?' % ' '.join([word[0] for word in s])

In [25]:
def make_questions(s):
    SourceQs = []
    WhyQs = []
    SubQs = []
    WhereQs = []
    AniQs = []
    WhenQs = []
    NEQs = []
    SpecQs = []
    switch_you_me = you_and_me(s)
    tagged_sent = return_tags(switch_you_me)
    split_matrix_sent = separate_matrix(tagged_sent)
    for s_ in [s for s in split_matrix_sent if not s in [[], None]]:
        ands_removed = split_at_and(s_)
        for s__ in [s for s in ands_removed if not s in [[], None]]:
            simplified_nested = simplify_nested(s__)
            partial_sentences = removal(simplified_nested)
            for s___ in [s for s in partial_sentences if not s in [[], None]]:
                removed_conjs = remove_conj(s___)
                changed_pronouns = change_view(removed_conjs)
                source_q = ask_source(changed_pronouns)
                SourceQs.append(source_q)
                added_aux = set_verb(changed_pronouns)
                reversed_subj = reverse_subj_aux(added_aux)
                why_q = why(reversed_subj)
                WhyQs.append(why_q)
                if not any(set(time_list) & set([word[4] for word in tagged_sent] + [word[2] for word in tagged_sent])):
                    when_q = when(reversed_subj)
                    WhenQs.append(when_q)
                where_q = where(reversed_subj)
                WhereQs.append(where_q)
                sub_q = subcategorization(reversed_subj)
                SubQs.append(sub_q)
                ne_q = ne_questions(reversed_subj)
                NEQs.append(ne_q)
                ani_q = animacy(reversed_subj)
                AniQs.append(ani_q)
                speco_q = specify_obj(reversed_subj)
                spec_q = specify_subj(changed_pronouns)
                SpecQs.append(spec_q)
                SpecQs.append(speco_q)
    return SubQs, WhereQs, WhyQs, WhenQs, AniQs, SpecQs, SourceQs, NEQs

In [26]:
def ask_random_q(s):
    questions = make_questions(s)
    qs = flatten(questions)
    Qs = [q for q in qs if q is not None]
    i = random.randint(0,len(Qs)-1)
    return qs[i]

In [27]:
def flatten(S):
    S = list(S)
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

In [28]:
# ONEA 1
# target question: 'Which man is eating?'
qs = make_questions('A certain man is eating.')
flatten(qs)

['Where is the certain man eating?',
 'Why is the certain man eating?',
 'When is the certain man eating?',
 'Which man is eating?',
 'How do you know that a certain man is eating?']

In [29]:
# ONEA 2
# target questions: 'What is he eating?', 'Why is he eating?', 'What else do we know about Peter?'
qs = make_questions('Peter is eating.')
flatten(qs)

['Where is Peter eating?',
 'Why is Peter eating?',
 'When is Peter eating?',
 'What else do we know about Peter?',
 'How do you know that Peter is eating?',
 'When did Peter live?',
 "What is Peter 's real name?",
 "What is Peter 's full name?",
 'What was Peter‘s real name?',
 'Where did Peter grow up?',
 'How long has Peter been around?',
 'When is Peter‘s birthday?',
 'Who is Peter?',
 'When was Peter born?',
 'Where did Peter come from?',
 "What is Peter 's address?",
 'Who are Peter‘s parents?',
 'What ethnicity is Peter?',
 'Where did Peter grow up?',
 'What nationality is Peter?',
 "What 's Peter 's real name?",
 'Where did Peter live?',
 'Where was Peter born?',
 'Where did Peter go to school?',
 'What country did Peter hail from?',
 'What college did Peter go to?',
 'What does Peter mean?',
 'How strong is Peter?',
 'Where was Peter born?',
 'Where does the name Peter come from?',
 "When is Peter 's Birthday?",
 'Where does Peter live , what state and city?',
 "What is Peter 

In [30]:
# ONEA 3
# target question: 'What is steel strong enough for?'
qs = make_questions('Steel is strong enough.')
flatten(qs)

['Why is steel strong?',
 'When is steel strong?',
 'Which steel is strong?',
 'How do you know that steel is strong?']

In [31]:
# ONEA 4
# target questions:  'Did he have enough?', 'Where do you know that from?', 'Did he use a spoon?'
qs = make_questions('Peter has eaten.')
flatten(qs)[:6]

['What has Peter eaten?',
 'Where has Peter eaten?',
 'Why has Peter eaten?',
 'When has Peter eaten?',
 'What else do we know about Peter?',
 'How do you know that Peter has eaten?']

In [32]:
# ONEA 5
# target question: 'What else do we know about the man?'
qs = make_questions('A man is eating.')
flatten(qs)

['Where is the man eating?',
 'Why is the man eating?',
 'When is the man eating?',
 'Which man is eating?',
 'How do you know that a man is eating?']

In [33]:
# ONEA 6
# target question: 'What else do we know about John?'
qs = make_questions('John saw a chair.')
flatten(qs)[:5]

['Why did John see the chair?',
 'When did John see the chair?',
 'What else do we know about John?',
 'Which chair did John see?',
 'What is the chair about?']

In [34]:
# ONEA 7
# target question: 'What is in the bag?'
qs = make_questions('Peter is eating sitting on a bag.')
flatten(qs)[:6]

['Where is Peter eating sitting on the bag?',
 'Why is Peter eating sitting on the bag?',
 'When is Peter eating sitting on the bag?',
 'What else do we know about Peter?',
 'Which bag is Peter eating sitting on?',
 'What is the bag about?']

In [35]:
# ONEA 8
# target question: 'What is the puzzle about?'
qs = make_questions('Peter is solving a puzzle.')
flatten(qs)[:6]

['Why is Peter solving the puzzle?',
 'When is Peter solving the puzzle?',
 'What else do we know about Peter?',
 'Which puzzle is Peter solving?',
 'What is the puzzle about?',
 'How do you know that Peter is solving a puzzle?']

In [36]:
# ONEA 9
# preferred target question: 'Which friendly girl does Peter love?'
qs = make_questions('Peter loves a friendly girl.')
flatten(qs)[:5]

['Why does Peter love the friendly girl?',
 'When does Peter love the friendly girl?',
 'What else do we know about Peter?',
 'Which friendly girl does Peter love?',
 'What is the friendly girl about?']

In [37]:
# ONEA 10
# preferred target question: 'Is the sandwich with ham?'
qs = make_questions('Peter is eating a sandwich.')
flatten(qs)[:7]

['Where is Peter eating the sandwich?',
 'Why is Peter eating the sandwich?',
 'When is Peter eating the sandwich?',
 'What else do we know about Peter?',
 'Which sandwich is Peter eating?',
 'What is the sandwich about?',
 'How do you know that Peter is eating a sandwich?']

In [38]:
# OWN EXAMPLE 1
# show better example of missing argument recognition, pronoun change
qs = make_questions('My sister was reading.')
flatten(qs)

['What was your sister reading?',
 'Why was your sister reading?',
 'When was your sister reading?',
 'How do you know that your sister was reading?']

In [39]:
# OWN EXAMPLE 2
# show sentence splitting, question generation with Named Entities, pronoun change
qs = make_questions('When I was younger, I really wanted to go to South America.')
flatten(qs)

['Why did you want to go to South America?',
 'Which South America America did you want to go to?',
 'How do you know that when you were younger?',
 'How do you know that you wanted to go to South America?',
 'What is the largest city in South America?',
 'How many people live in South America?']

In [40]:
# OWN EXAMPLE 3
# show subject recognition
qs = make_questions('The fat friendly old white man was waiting.')
flatten(qs)

['What was the fat friendly old white man waiting?',
 'What was the fat friendly old white man waiting for?',
 'Who was the fat friendly old white man waiting for?',
 'Why was the fat friendly old white man waiting?',
 'When was the fat friendly old white man waiting?',
 'How do you know that the fat friendly old white man was waiting?']

In [41]:
# SNOWDEN 1
# target question: 'Why did you ask for police protection?'
qs = make_questions('Because, I was reading that you asked for a kind of police protection.')
flatten(qs)

['Why were you reading that you asked for the kind of police protection?',
 'When were you reading that you asked for the kind of police protection?',
 'Which that were you reading?',
 'Which that you asked for a kind were you reading?',
 'Which that you asked for a kind of police protection were you reading?',
 'How do you know that you were reading that you asked for a kind of police protection?']

In [42]:
# SNOWDEN 1_2
qs = make_questions('Because, I read that you asked for a kind of police protection.')
flatten(qs)

['What did you read?',
 'What did you read?',
 'Why did you read?',
 'Why did you ask for the kind of police protection?',
 'When did you read?',
 'When did you ask for the kind of police protection?',
 'Which kind did you ask for?',
 'What is the kind about?',
 'Which kind of police protection did you ask for?',
 'What is the kind of police protection about?',
 'How do you know that you read?',
 'How do you know that you asked for a kind of police protection?']

In [43]:
# SNOWDEN 2
# target question: 'What kind of threat?'
qs = make_questions('There are significant threats but I sleep very well.')
flatten(qs)

['What do you sleep?',
 'Why are there significant threats?',
 'Why do you sleep?',
 'When are there significant threats?',
 'When do you sleep?',
 'Which there are significant threats?',
 'How do you know that there are significant threats?',
 'How do you know that you sleep?']

In [44]:
# SNOWDEN 3
# target question: 'What kind of article?'
qs = make_questions('There was an article that came out in an online outlet called BuzzFeed.')
flatten(qs)

['What was there the article that did come out in the online outlet called BuzzFeed on?',
 'Who was there the article that did come out in the online outlet called BuzzFeed on?',
 'Why was there the article that did come out in the online outlet called BuzzFeed?',
 'When was there the article that did come out in the online outlet called BuzzFeed?',
 'What else do we know about BuzzFeed?',
 'Which there was an article that come out in an online outlet called BuzzFeed?',
 'Which there was an article that come out in an online outlet called BuzzFeed?',
 'Which online outlet was there an article that did come out in?',
 'What is the online outlet about?',
 'How do you know that there was an article that came out in an online outlet called BuzzFeed?',
 'What year did BuzzFeed begin?',
 'Who is the CEO of BuzzFeed?',
 'What year was BuzzFeed founded?',
 'What in the world is BuzzFeed?',
 'What does BuzzFeed stand for?',
 'Who is in charge of BuzzFeed?',
 'What is the meaning of BuzzFeed?',


In [45]:
# SNOWDEN 4
# target question: 'How did they interview them?
qs = make_questions('They interviewed officials from the Pentagon, from the National Security Agency.')
flatten(qs)

['Why did they interview officials from the Pentagon?',
 'When did they interview officials from the Pentagon?',
 'What else do we know about them?',
 'What else do we know about Pentagon?',
 'Which officials did they interview?',
 'Which officials from the Pentagon did they interview?',
 'How do you know that they interviewed officials from the Pentagon?',
 'What year did Pentagon begin?',
 'Who is the CEO of Pentagon?',
 'What year was Pentagon founded?',
 'What in the world is Pentagon?',
 'What does Pentagon stand for?',
 'Who is in charge of Pentagon?',
 'What is the meaning of Pentagon?',
 'Where is Pentagon located ?',
 'What does Pentagon stand for?',
 'When was Pentagon founded?',
 'What does Pentagon mean?']

In [46]:
# SNOWDEN 5
# target question: 'So what did they say?'
qs = make_questions('They gave them anonymity to be able to say what they wanted.')
flatten(qs)

['Why did they give them anonymity to be able to say what they wanted?',
 'When did they give them anonymity to be able to say what they wanted?',
 'What else do we know about them?',
 'What else do we know about them?',
 'What else do we know about them?',
 'Which anonymity to be able to say what they wanted?',
 'Which anonymity did they give them?',
 'Which anonymity to be able to say what did they give them?',
 'How do you know that they gave them anonymity to be able to say what they wanted?']

In [48]:
# SNOWDEN 6
# target question: 'Who said that?'
qs = make_questions('What they told the reporter was that they wanted to murder me.')
flatten(qs)

['What what did they tell the reporter was?',
 'Why did they want to murder you?',
 'When what did they tell the reporter was?',
 'When did they want to murder you?',
 'What else do we know about them?',
 'What else do we know about them?',
 'How do you know that what they told the reporter was?',
 'How do you know that they wanted to murder you?']

In [49]:
# SNOWDEN 7
# target question: 'What did they say, how would they want to murder me?'
qs = make_questions('These individuals are acting government officials.')
flatten(qs)

['Why are these individuals acting government officials?',
 'When are these individuals acting government officials?',
 'Which government officials are these individuals acting?',
 'How do you know that these individuals are acting government officials?']

In [50]:
# SNOWDEN 8
# target question: 'In what other way would they like to murder me?'
qs = make_questions('They said they would be happy, they would love to put a bullet in my head.')
flatten(qs)

['What did they say they would be happy?',
 'What did they say they would be happy?',
 'What would they love to put the bullet in your head into?',
 'Who would they love to put the bullet in your head into?',
 'Why did they say they would be happy?',
 'Why would they love to put the bullet in your head?',
 'When did they say they would be happy?',
 'When would they love to put the bullet in your head?',
 'What else do we know about them?',
 'What else do we know about them?',
 'What else do we know about them?',
 'Which bullet would they love to put?',
 'What is the bullet about?',
 'Which bullet in your head would they love to put?',
 'What is the bullet in your head about?',
 'How do you know that they said they would be happy?',
 'How do you know that they would love to put a bullet in your head?']

In [51]:
# SNOWDEN 9
# target question: 'What would be the result of being poisoned like that?'
qs = make_questions('They would love to poison me as I was returning from the grocery store.')
flatten(qs)

['How were you returning from the grocery store?',
 'What were you returning from the grocery store to?',
 'Who were you returning from the grocery store to?',
 'Why would they love to poison you?',
 'Why were you returning from the grocery store?',
 'When would they love to poison you?',
 'When were you returning from the grocery store?',
 'What else do we know about them?',
 'How do you know that they would love to poison you?',
 'How do you know that you were returning from the grocery store?']

In [52]:
# SNOWDEN 10
# target question: 'And what else can be said about your wellbeing?'
qs = make_questions('Right, but I am still alive.')
flatten(qs)

['Why are you alive?',
 'When are you alive?',
 'How do you know that you are alive?']

In [53]:
# SNOWDEN 11
# target question: 'Why is that?'
qs = make_questions('and I do not lose sleep.')
flatten(qs)

['Why do you not lose sleep?',
 'When do you not lose sleep?',
 'Which sleep do you not lose?',
 'How do you know that you do not lose sleep?']

In [54]:
# SNOWDEN 12
# target question: 'Why did you feel that way?'
qs = make_questions('because I have done what I feel I needed to do.')
flatten(qs)

['Why have you done what you feel you needed to do?',
 'When have you done what you feel you needed to do?',
 'Which what have you done?',
 'How do you know that you have done what you feel you needed to do?']